# Analyze Scan/IRC jobs

This notebook is used for analyzing scan/IRC jobs from quantum chemsitry calculation. 
This notebook assumes the scan to be one dimensional rotor scan.

In [1]:
import os
import re
import sys
# To add this RDMC into PYTHONPATH in case you haven't do it
sys.path.append(os.path.dirname(os.path.abspath('')))

import cclib
import numpy as np
import py3Dmol
import matplotlib.pyplot as plt

from rdmc.mol import RDKitMol
from rdmc.view import mol_viewer, mol_animation

from ipywidgets import interact, fixed, IntSlider

# atom index starting from 0 or 1
ATOMINDEX = {'gaussian': 1, }

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Load the file

In [2]:
log = "/Users/xiaorui/Dropbox (Personal)/RMG/Co-OPTIMA shared/relax-rotor/Species/[O][CH]CC=O/scan_5_3_4_1/output.out"
######################################
# Parse the output file
calc_results = cclib.io.ccread(log)
if not hasattr(calc_results, 'metadata'):
    raise RuntimeError('The cclib cannot path the file!')
else:
    package = calc_results.metadata['package'].lower()

# Check success
if not calc_results.metadata['success']:
    print('The job is not successful.')

# Check job type
if hasattr(calc_results, 'scancoords'):
    job_type = 'scan'
    print(f'Scanning: {", ".join(calc_results.scannames)}')
    # Use RDKitmol to load the geometries
    mol = RDKitMol.FromXYZ(cclib.io.ccwrite(calc_results, outputtype='xyz',
                       indices=0, returnstr=True,))
    mol_viewer(mol.ToMolBlock(), 'sdf').update()
else:
    job_type = 'irc'
    mol_viewer(cclib.io.ccwrite(calc_results, outputtype='xyz',
                                indices=0, returnstr=True,), 'xyz').update()


Scanning: D(5,3,4,1)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 2. Extract all converged intermediate xyzs

In [3]:
# Find all geometries meet convergence criteria 
# - fulfill at least 2 criteria for gaussian)
good_geo_idx = np.arange(calc_results.optstatus.shape[0])[calc_results.optstatus >= 2]
good_geo_scf = calc_results.scfenergies[good_geo_idx]
num_conf = good_geo_scf.shape[0]

# find inverse point
if job_type == 'irc':
    # An IRC job can be either unidirectional or bidirectional
    # If bidirectional, there will be a transition going back to the TS in Gaussian IRC
    scf_diff = np.diff(good_geo_scf)
    if np.alltrue(scf_diff < 0):  # no transition
        print('This is a unidirectional IRC scan!')
        starting_pt = 0  # Starting from the first geometry when plotting
    else:
        iv_pt = np.argmax(scf_diff)  # get the inversion point
        good_geo_idx[0:iv_pt+1] = good_geo_idx[iv_pt::-1]  # reverse the sequence before the inversion point
        good_geo_scf = calc_results.scfenergies[good_geo_idx]
        starting_pt = iv_pt
        print('This is a bidirectional IRC scan!')
    # Get xyzs
    xyzs = [cclib.io.ccwrite(calc_results, outputtype='xyz',
                             indices=i, returnstr=True,)
            for i in good_geo_idx]

elif job_type == 'scan':
    starting_pt = 0
    
    # Use RDKit mol for mol manipulation
    # Mapping the first 3 atoms of the scanned torsional internal coordinate for better visualization experience
    mol.EmbedMultipleConfs(num_conf)
    
    try:
        # Force angles to be 0 - 2pi with the initial angle = 0
        angles = np.array(calc_results.scanparm[0]) - calc_results.scanparm[0][0]
        angles[angles < 0] += 2 * np.pi
    
        scanname = [int(i) for i in re.findall(r'\d+', calc_results.scannames[0])]
        atom_map = [(int(i)-ATOMINDEX[package], int(i)-ATOMINDEX[package]) 
                    for i in re.findall(r'\d+', calc_results.scannames[0])[:3]]
        for scan_idx, geo_idx in enumerate(good_geo_idx):
            mol.SetPositions(calc_results.atomcoords[geo_idx], scan_idx)
            if scan_idx != 0:
                mol.AlignMol(refMol=mol, prbCid=int(scan_idx),
                             refCid=0, atomMap=atom_map)
    except:
        pass
    xyzs = [mol.ToXYZ(confId=i) for i in range(num_conf)]

## 3. Visualize the trajectory animation
Note: The atom labels are not moving with the corresponding atoms. Need further investigations into 3Dmol to solve this issue.

In [4]:
multiple_xyzs = "".join(xyzs)
viewer = mol_animation(multiple_xyzs, 'xyzs')
viewer.update()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 4. Interact with intermediate conformers

In [5]:
energies = (good_geo_scf - np.min(good_geo_scf)) * 627.5 / 27.211
def visualize_trajectory(idx):
    xyz = xyzs[idx]
    mol_viewer(xyz, model='xyz').update()
    ax = plt.axes()
    ax.grid(True)
    ax.plot(energies, 'o', color='tab:blue')
    ax.plot([idx], [energies[idx]], 'ro')
    ax.set_ylabel('kcal/mol')
    plt.show()
    print(xyzs[idx])
    
    
int_slider = IntSlider(value=starting_pt,
                       min=0, max=num_conf-1, step=1,
                       description='Index',)

interact(visualize_trajectory, 
         idx=int_slider)


interactive(children=(IntSlider(value=0, description='Index', max=45), Output()), _dom_classes=('widget-intera…

<function __main__.visualize_trajectory(idx)>